In [1]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, plot_confusion_matrix, make_scorer
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from matplotlib import pyplot as plt
from sklearn.metrics import mean_squared_error

# Read Data

In [2]:
# GALAXY DATASET
def DatasetGALAXY():
    df = pd.read_csv('../../Data/galaxymorphology/dataset1_sydney.csv')
#    df.head()
#    df['target'].value_counts()
#     mapping = {
#         'elliptical': 0,
#         'spiral':1
#     }
#     df['target'] = df['target'].map(mapping)
    x = df.sample(df.shape[0])
    df = x.iloc[:, :5]
    df['target'] = x.iloc[:,-1]
    return df

df = DatasetGALAXY()
header = df.columns

In [47]:
#CAR DATASET
def DatasetCAR():
    df = pd.read_csv('../../Data/dataset1-car/car_dataset.csv')
    
    #     df['buying'].replace(['vhigh', 'high', 'med', 'low'],[3,2,1,0], inplace = True)
    #     df['maintenance'].replace(['vhigh', 'high', 'med', 'low'],[3,2,1,0], inplace = True)
    #     df['doors'].replace(['2', '3', '4', '5more'],[2,3,4,5], inplace = True)
    #     df['persons'].replace(['2', '4', 'more'],[2,4,5], inplace = True)
    #     df['lug_boot'].replace(['small', 'med', 'big'],[1,2,3], inplace = True)
    #     df['safety'].replace(['high', 'med', 'low'],[3,2,1], inplace = True)
    #     df['target'].replace(['unacc', 'acc', 'good', 'vgood'],[0,1,2,3], inplace = True)
    #     df= df[df.target !=3]
    #     df= df[df.target !=2]
    print(df['target'].nunique())
    df = df.sample(df.shape[0])
    return df

df = DatasetCAR()
header = df.columns

4


In [123]:
#IRIS DATASET
def DatasetIRIS():
    df = pd.read_csv('../../Data/iris/iris.data')
    df['target'].replace(['Iris-setosa', 'Iris-versicolor', 'Iris-virginica'],[0,1,2], inplace = True)
    df = df.sample(df.shape[0])
    return df
df = DatasetIRIS()
header = df.columns

In [126]:
#TITANIC DATASET
def DatasetTITANIC():
    df = pd.read_csv('../../Data/Titanic/train.csv')
    df.drop(columns = [ 'Cabin','Embarked', 'Ticket','Name','PassengerId'], inplace=True)
    df['Sex'].replace(['male', 'female'],[0,1], inplace = True)
    df =df.dropna()
    
    return df
df = DatasetTITANIC()
header = df.columns

In [3]:
df

,u-g,g-r,r-i,i-z,ecc,target
457,1.58693,0.76100,0.40371,0.30242,0.902340,spiral
308,1.51834,0.64705,0.34067,0.24506,0.858611,spiral
447,1.90540,0.77048,0.38798,0.23775,0.818126,spiral
221,1.94276,0.87164,0.42491,0.30789,0.938287,elliptical
106,1.78496,0.77980,0.37753,0.28232,0.829166,elliptical
...,...,...,...,...,...,...
70,1.78528,0.94088,0.37692,0.36588,0.794977,elliptical
28,1.88772,0.85597,0.41393,0.32151,0.759583,elliptical
66,1.97871,1.02908,0.41819,0.34289,0.717513,elliptical
254,1.93122,1.06486,0.43941,0.36051,0.864887,elliptical


# Decision Trees

In [ ]:
"""
We took help from the internet for the equations, and understanding how to apply some methods into coding format
"""

In [5]:
#Counts the number of each type of example in a dataset
def class_counts(rows):
    counts = {}  # a dictionary of label -> count.
    for row in rows:
        label = row[-1]
        if label not in counts:
            counts[label] = 0
        counts[label] += 1
    return counts

In [6]:
def is_numeric(value):
    return isinstance(value, int) or isinstance(value, float)

In [7]:
# the splitting question
class Question:

    def __init__(self, column, value):
        self.column = column #1
        self.value = value #3
    
    #Check if a row answers the question
    def match(self, example): 
        val = example[self.column]
        if is_numeric(val):
            return val >= self.value
        else:
            return val == self.value
    
    # prints the question
    def __repr__(self):
        condition = "=="
        if is_numeric(self.value):
            condition = ">="
        return "Is %s %s %s?" % (
            header[self.column], condition, str(self.value))


In [8]:
# splits the dataset according to the question
# true rows satisfy the question 
def partition(rows, question):
    true_rows, false_rows = [], []
    for row in rows:
        if question.match(row):
            true_rows.append(row)
        else:
            false_rows.append(row)
    return true_rows, false_rows

In [9]:
def gini(rows):
    """Calculate the Gini Impurity for a list of rows.
    https://en.wikipedia.org/wiki/Decision_tree_learning#Gini_impurity
    """
    counts = class_counts(rows)
    impurity = 1
    for label in counts:
        prob_of_label = counts[label] / float(len(rows))
        impurity -= prob_of_label**2
    return impurity

In [10]:
def info_gain(left, right, current_uncertainty):
    """Information Gain.gini(df)
    The uncertainty of the starting node, minus the weighted impurity of
    two child nodes.
    """
    p = float(len(left)) / (len(left) + len(right))
    return current_uncertainty - p * gini(left) - (1 - p) * gini(right)

In [11]:
def find_best_split(rows):
    """Find the best question to ask by iterating over every feature / value
    and calculating the information gain."""
    best_gain = 0  # keep track IG
    best_question = None  # keep track of the feature / value that produced it
    current_uncertainty = gini(rows)
    n_features = len(rows[0]) - 1  # number of columns

    for col in range(n_features):  # for each feature

        values = set([row[col] for row in rows])  # unique values in the column

        for val in values:  # for each value

            question = Question(col, val)

            true_rows, false_rows = partition(rows, question)

            # Skip split if it doesn't divide the dataset.
            if len(true_rows) == 0 or len(false_rows) == 0:
                continue

            # Calculate the information gain from this split
            gain = info_gain(true_rows, false_rows, current_uncertainty)

            if gain >= best_gain:
                best_gain, best_question = gain, question

    return best_gain, best_question

In [12]:
class Leaf:
    """A Leaf node classifies data.
    """

    def __init__(self, rows):
        self.predictions = class_counts(rows)


In [13]:
class Decision_Node:
    """A Decision Node asks a question.
    This holds a reference to the question, and to the two child nodes.
    """

    def __init__(self, question, true_branch, false_branch):
        self.question = question
        self.true_branch = true_branch
        self.false_branch = false_branch

In [14]:
def build_tree(rows):
    """Builds the tree.
    """

    # calculate the information gain,
    gain, question = find_best_split(rows)

    if gain == 0:
        return Leaf(rows)


    true_rows, false_rows = partition(rows, question)

    # Recursively build the true branch.
    true_branch = build_tree(true_rows)

    # Recursively build the false branch.
    false_branch = build_tree(false_rows)

    # Return a Question node.
    return Decision_Node(question, true_branch, false_branch)


In [15]:
def print_tree(node, spacing=""):

    # Base case: we've reached a leaf
    if isinstance(node, Leaf):
        print (spacing + "Predict", node.predictions)
        return

    # Print the question at this node
    print (spacing + str(node.question))

    # Call this function recursively on the true branch
    print (spacing + '--> True:')
    print_tree(node.true_branch, spacing + "  ")

    # Call this function recursively on the false branch
    print (spacing + '--> False:')
    print_tree(node.false_branch, spacing + "  ")

In [16]:
def classify(row, node):

    # leaf
    if isinstance(node, Leaf):
        return node.predictions

    # Decide whether to follow the true-branch or the false-branch.
    if node.question.match(row):
        return classify(row, node.true_branch)
    else:
        return classify(row, node.false_branch)

In [17]:
def print_leaf(counts):
    """A nicer way to print the predictions at a leaf."""
    total = sum(counts.values()) * 1.0
    probs = {}
    for lbl in counts.keys():
        probs[lbl] = str(int(counts[lbl] / total * 100)) + "%"
        
    return probs

In [18]:
def get_overall_cm(df):
    classes = df['target'].nunique()
    if classes < 2:
        classes = 2
    overall_cm = []
    for i in range(classes):
        overall_cm.append( [0] * classes)
    return overall_cm

In [19]:
def Kfold(df,n_folds, test_size, overall_cm):
    # Iterate through all folds
    predicted= []
    actual = []
    for i in range(n_folds):
        print("\n\n\n\nTHIS IS FOLD:", i)
        #split test and train
        test = df.iloc[test_size * i : test_size*i+test_size].to_numpy()
        train = df.iloc[test_size:].to_numpy()

        ####### THIS IS WHERE YOU CHANGE THE ALG #######
        #start training and testing the dataset
        my_tree = build_tree(train)
        print_tree(my_tree)
        
        
        for row in test:
            most_probable = print_leaf(classify(row, my_tree))
            predicted += [next(iter(most_probable))]
            actual += [row[-1]]
            print("Actual: %s. Predicted: %s" %(row[-1], print_leaf(classify(row, my_tree))))
        # generate and append cm
        overall_cm = overall_cm + confusion_matrix(actual, predicted)
    print(classification_report(actual, predicted ))
    return overall_cm


In [20]:
if __name__ == "__main__":
    # KFOLD take in n folds and the size of dataset
    
    # DATASET1 SYDNEY
    n_folds = 2
    test_size = int(df.shape[0]/n_folds)

    # get overall cm returns a Confusion matrix with 0's shape depends on (classes)
    dtcm = Kfold(df,n_folds, test_size, get_overall_cm(df))
    print(f"\n\n\n Confusion Matrix\n{dtcm}")





THIS IS FOLD: 0
Is u-g >= 1.7402400000000002?
--> True:
  Is g-r >= 0.79406?
  --> True:
    Is u-g >= 1.831669999999999?
    --> True:
      Is ecc >= 0.7773678?
      --> True:
        Predict {'elliptical': 69}
      --> False:
        Is ecc >= 0.7754218?
        --> True:
          Predict {'spiral': 1}
        --> False:
          Is i-z >= 0.3125599999999995?
          --> True:
            Predict {'elliptical': 22}
          --> False:
            Is i-z >= 0.3114999999999988?
            --> True:
              Predict {'spiral': 1}
            --> False:
              Predict {'elliptical': 10}
    --> False:
      Is r-i >= 0.42051000000000016?
      --> True:
        Is i-z >= 0.3685700000000001?
        --> True:
          Predict {'elliptical': 1}
        --> False:
          Predict {'spiral': 2}
      --> False:
        Is ecc >= 0.6293741?
        --> True:
          Is i-z >= 0.34914999999999985?
          --> True:
            Is ecc >= 0.7949773?
            --

In [66]:

# if __name__ == '__main__':
    
#     my_tree = build_tree(sydney_train)

#     print_tree(my_tree)

#     predicted= []
#     actual = []
#     for row in sydney_test:
#         most_probable = print_leaf(classify(row, my_tree))
#         predicted += [next(iter(most_probable))]
#         actual += [row[-1]]
#         print("Actual: %s. Predicted: %s" %(row[-1], print_leaf(classify(row, my_tree))))
#     print(classification_report(actual, predicted))